## Dynamic Scraping with Selenium and Chrome webdriver

This is a simple notebook to work with Selenium and Chrome Webdriver in order to scrape a dynamic website. While for static websites we can use [Beautiful Soup](https://pypi.org/project/beautifulsoup4/), when we are working with dynamic ones we need Selenium. The key aspect of [Selenium](https://pypi.org/project/selenium/) is that it executes javascript when loading a page request, this allows for dynamic websites to be scraped easily. To complete our exercise we will also use [pandas](https://pypi.org/project/pandas/), the data science and data analysis library, in order to create a dataframe and export our data to our local machine.

**Tutorial sections:**
- **Headless browsers**: Start a WebDriver session with Selenium
- **One page simple scraping**: retrieve all the content from one page and create a dataframe
- **Scraping across pages**: Continously load new content, then create a dataframe

**Step 1: import**

In [73]:
# Will make scraping possible
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time

# Will allow to request, access and write urls of images and other links
import urllib.request
import requests

# Will provide structure to data
import pandas as pd

Initialize Selenium and driver instance using Chrome WebDriver. To properly install ChromeDriver you can manually set the path following [this guide](https://www.kenst.com/2015/03/including-the-chromedriver-location-in-macos-system-path/) or use `pip install webdriver-manager` to manage it.

**Step 2: start Chrome webdriver session**

For the purpose of this exercise we will get some movies from [Rotten Tomatoes](https://www.rottentomatoes.com/browse/dvd-streaming-all). We are interested in creating a dataset of titles with their ranking, date of release and cover.

In [112]:
# We start an headless Chrome session. (Headless means that we don't open a visual browser window)
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

# instead of setting path manually (still possible but a bit annoying) we use DriverManager
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)
driver.get("https://www.rottentomatoes.com/browse/dvd-streaming-all")

# print(driver.page_source)
print('done')

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/francescamorini/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


 
done


### One time simple scraping

The concept behind scraping is quite easy to grasp. We are creating a script that behaves like a human. By using the correct methods we can fake human interactions on webpage. While usually scripts "see" different things than us, Selenium, Beautiful Soup and WebDrivers are built to see pages as we would do. Scripts are obviously faster, they don't get to go to the toilet and they do everything very enthusiastically.

**Step 3: Get movies**

In [49]:
# Only getting first page results
movies = driver.find_elements_by_class_name('mb-movie')

**Step 4: loop through movies and build a list of lists containing our data**

In [114]:
# This empty list will later contain my precious data
moviesData = []
# I am creating a counter, this will print the parser status and generate filenames for images
count = 0
for movie in movies:
    
    count = count + 1
    print(count)
    
    # we can get precise elements by using css selectors, classes, tag names and xpaths
    title = movie.find_element_by_class_name('movieTitle').text
    rating = movie.find_element_by_class_name('tMeterIcon').text
    available = movie.find_element_by_class_name('release-date').text
    image = movie.find_element_by_tag_name('img')
    
    # it is also possible to get accessory information from collected elements, 
    # in this case we are interested in the src attribute
    source = image.get_attribute("src")
    
    # We use urllib to read the poster images and save them locally in a special folder. In order to avoid
    # overwriting we use the {0}.format('dynamic variable') to save all of them
    imgdestination = "data/img/poster-{0}.jpg".format(count)
    urllib.request.urlretrieve(source, imgdestination)
    
    # We append a shorter list to the initial list
    moviesData.append([title, rating, available, imgdestination])

# Safety print to check everything is fine!
print(moviesData)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


**Step 5: Move list of lists content to a pandas dataframe**

We use pandas to create a dataframe holding the information we gathered from Rotten Tomatoes. This is the gateway to data analysis and data cleaning. From pandas we can export a `.csv` file that can later be cleaned, loaded in various applications and finally used to code our prototype.

In [115]:
AllStreamingTitles = pd.DataFrame(moviesData, columns=["title", "rating", "available", "img"])

In [116]:
#Previewing 5 random rows from my dataframe
AllStreamingTitles.sample(5)

,title,rating,available,img
271,Tijuana Jackson: Purpose Over Prison,67%,Available Jul 31,data/img/poster-272.jpg
292,Dead Dicks,100%,Available Jul 28,data/img/poster-293.jpg
167,Entwined,65%,Available Sep 8,data/img/poster-168.jpg
212,Hard Kill,0%,Available Aug 25,data/img/poster-213.jpg
113,The Secrets We Keep,64%,Available Oct 16,data/img/poster-114.jpg


**Step 6: Export dataframe to csv**

In [117]:
# Check the 'data' subfolder after executing this cell to see the file.
AllStreamingTitles.to_csv('data/titlesDataRaw.csv', sep=',')

### Scraping across multiple pages 

What if we need to scrape more than the first page? Currently we are getting titles only from the initial screen. 
Instead we want to keep going: our script should be able to proceed down the page and do something everytime new titles need to be loaded. To approach this problem we need to go back to the source website. At the bottom of the page it's possible to spot a yellow "Show More" button, if we click on it new movies are loaded. 

**Positive aspects**
- We have a distinct element, the yellow button with fairly easy interaction
- We have a stable pattern and everything is happening on one single page, we don't need to touch the page URL
- We can adopt a similar procedure to the previous one.

**Negative aspects**
- The page indefinitely loads new content, the button does not change and/or disappear at some point

This calls for a fairly simple strategy. We will to modify only one step of our procedure: step 3. Instead of creating a variable with the first page results, we will create a while loop that keeps loading content up to a certain point, while getting all the movies. 

**Step 1 and 2 stay the same!**

**Step 3: automate clicking of the "Show More" button**

Websites generally don't like to be scraped (on some websites it's explicitly forbidden, so be careful). It can happen that if we are requesting too many pages or if we are interacting with the page too quickly we might get blocked or throttled (slowed). We will fake interactions with the "Show More" button every 10 seconds to avoid throttle.

In [119]:
pageExpandLimit = 0

while True:
    # Add 1 to counter so we can artificially stop the loop
    print(pageExpandLimit)
    pageExpandLimit = pageExpandLimit + 1 
    
    # Taking a screenshot everytime we click on the button (to check where my browser's at)
    driver.save_screenshot("screenshot{0}.png".format(pageExpandLimit))
    
    if pageExpandLimit < 10:
        
        # Generic asleep time to avoid throttle
        time.sleep(5)
        # Find the button element and click on it to keep loading movies
        driver.find_element_by_class_name('mb-load-btn').click()  
        # Dump all the elements in one list
        moreMovies = driver.find_elements_by_class_name('mb-movie')
        
    # It will set the condition to False if counter is greater than 5, this will break the while loop
    if pageExpandLimit > 10:
        break

0
1
2
3
4
5
6
7
8
9
10


**Steps 4, 5 and 6 are identical to the "one page simple scraping" example**

In [120]:
#Ultimately, once we got everything we were interested in, we need to quit our WebDriver to avoid sessions piling up
driver.quit()

### Conclusion

Each website is different, therefore scraping is often a tailored activity that requires some coding and strategizing effort. This tutorial presented some simple features and a couple of useful patterns if you need to scrape a relatively contained amount of data. There are more complex and articulated tasks that require more savviness, time and computing effort. One example are social media. For special or recurring cases there's a useful collection of tools that can be found [here](https://wiki.digitalmethods.net/Dmi/ToolDatabase).

### References

1. Guide to Selenium basic functions for scraping: https://www.scrapingbee.com/blog/selenium-python/
2. Including WebDriver location in MacOS System Path: https://www.kenst.com/2015/03/including-the-chromedriver-location-in-macos-system-path/
3. Data Science Skills Set: https://francesco-ai.medium.com/data-science-skills-list-9f38863adab5
4. DMI Tools Database: https://wiki.digitalmethods.net/Dmi/ToolDatabase